# Курсовая работа по курсу:  "Рекомендательные системы" Трифонов Р.С.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import random

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys

import src

In [10]:
path_to_files = '/data'

In [11]:
data          = pd.read_csv(os.path.join(path_to_files, "data.csv"))
user_features = pd.read_csv(os.path.join(path_to_files, "user_features.csv"))
item_features = pd.read_csv(os.path.join(path_to_files, "item_features.csv"))

In [12]:
# схема обучения и валидации
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,999999,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,999999,1,0.82,364,0.0,1631,1,0.0,0.0


In [13]:
recommender = MainRecommender(data_train_lvl_1, item_features, user_features)

user_item_matrix


GPU training requires factor size to be a multiple of 32. Increasing factors from 50 to 64.


user_feat
item_feat
ALS model


ALS TF-IDF model


ALS BM25 model
LightFM model
Own


In [14]:
users_lvl_2_pred_als = pd.DataFrame(data_val_lvl_2['user_id'].unique())
users_lvl_2_pred_als.columns = ['user_id']

In [116]:
result_lvl_2 = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_2.columns=['user_id', 'actual']

In [117]:
k_n = 5
num = 40

In [118]:
popularity = data.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
item_list = popularity.sort_values('n_sold', ascending=False).head(num+1).item_id.tolist()
item_list = item_list[1:num+1]
len(item_list)

40

In [119]:
user_train_list = data_train_lvl_1["user_id"].unique().tolist()

In [19]:
def get_pred_als(recommender, user_train_list, item_list, x, num=5):
    res = []
    if x in user_train_list:
        res.extend(recommender.get_als_recommendations(x, rec_num=num))
    else:
        res.extend(item_list)    # res.extend(random.choices(item_list, k=num))
    return res

In [20]:
def get_pred_als_tfidf(recommender, user_train_list, item_list, x, num=5):
    res = []
    if x in user_train_list:
        res.extend(recommender.get_als_tfidf_recommendations(x, rec_num=num))
    else:
        res.extend(item_list)    # res.extend(random.choices(item_list, k=num))
    return res

In [21]:
def get_pred_als_bm25(recommender, user_train_list, item_list, x, num=5):
    res = []
    if x in user_train_list:
        res.extend(recommender.get_als_bm25_recommendations(x, rec_num=num))
    else:
        res.extend(item_list)    # res.extend(random.choices(item_list, k=num))
    return res

In [22]:
def get_pred_own(recommender, user_train_list, item_list, x, num=5):
    res = []
    if x in user_train_list:
        res.extend(recommender.get_own_recommendations(x, rec_num=num))
    else:
        res.extend(item_list)    # res.extend(random.choices(item_list, k=num))
    return res

In [23]:
def get_pred_similar_items(recommender, user_train_list, item_list, x, num=5):
    res = []
    if x in user_train_list:
        res.extend(recommender.get_similar_items_recommendation(x, rec_num=num))
    else:
        res.extend(item_list)    # res.extend(random.choices(item_list, k=num))
    return res

In [24]:
users_lvl_2_pred_als['predictions'] = users_lvl_2_pred_als['user_id'].apply(\
                            lambda x: get_pred_als(recommender, user_train_list, item_list, x, num=num))
    
users_lvl_2_pred_als["prec"] = users_lvl_2_pred_als['user_id'].apply(\
        lambda x: precision_at_k(users_lvl_2_pred_als.loc[users_lvl_2_pred_als["user_id"] == x, "predictions"].values[0], 
                                 list(result_lvl_2.loc[result_lvl_2["user_id"] == x, "actual"].values[0]), 
                                 k=k_n))

users_lvl_2_pred_als["prec"].mean()

0.2162585700293797

In [25]:
users_lvl_2_pred_als["rec"] = users_lvl_2_pred_als['user_id'].apply(\
        lambda x: recall_at_k(users_lvl_2_pred_als.loc[users_lvl_2_pred_als["user_id"] == x, "predictions"].values[0], 
                              list(result_lvl_2.loc[result_lvl_2["user_id"] == x, "actual"].values[0]), 
                              k=k_n))

users_lvl_2_pred_als["rec"].mean()

0.03958029471138852

In [26]:
users_lvl_2_pred_als['predictions_tfidf'] = users_lvl_2_pred_als['user_id'].apply(\
                            lambda x: get_pred_als_tfidf(recommender, user_train_list, item_list, x, num=num))
    
users_lvl_2_pred_als["prec_tfidf"] = users_lvl_2_pred_als['user_id'].apply(\
        lambda x: precision_at_k(users_lvl_2_pred_als.loc[users_lvl_2_pred_als["user_id"] == x, "predictions_tfidf"].values[0], 
                                 list(result_lvl_2.loc[result_lvl_2["user_id"] == x, "actual"].values[0]), 
                                 k=k_n))

users_lvl_2_pred_als["prec_tfidf"].mean()

0.16346718903036014

In [27]:
users_lvl_2_pred_als["rec_tfidf"] = users_lvl_2_pred_als['user_id'].apply(\
        lambda x: recall_at_k(users_lvl_2_pred_als.loc[users_lvl_2_pred_als["user_id"] == x, "predictions_tfidf"].values[0], 
                              list(result_lvl_2.loc[result_lvl_2["user_id"] == x, "actual"].values[0]), 
                              k=k_n))

users_lvl_2_pred_als["rec_tfidf"].mean()

0.031148511746959033

In [28]:
users_lvl_2_pred_als['predictions_bm25'] = users_lvl_2_pred_als['user_id'].apply(\
                            lambda x: get_pred_als_bm25(recommender, user_train_list, item_list, x, num=num))
    
users_lvl_2_pred_als["prec_bm25"] = users_lvl_2_pred_als['user_id'].apply(\
        lambda x: precision_at_k(users_lvl_2_pred_als.loc[users_lvl_2_pred_als["user_id"] == x, "predictions_bm25"].values[0], 
                                 list(result_lvl_2.loc[result_lvl_2["user_id"] == x, "actual"].values[0]), 
                                 k=k_n))

users_lvl_2_pred_als["prec_bm25"].mean()

0.13898139079333827

In [29]:
users_lvl_2_pred_als["rec_bm25"] = users_lvl_2_pred_als['user_id'].apply(\
        lambda x: recall_at_k(users_lvl_2_pred_als.loc[users_lvl_2_pred_als["user_id"] == x, "predictions_bm25"].values[0], 
                              list(result_lvl_2.loc[result_lvl_2["user_id"] == x, "actual"].values[0]), 
                              k=k_n))

users_lvl_2_pred_als["rec_bm25"].mean()

0.0256680778793162

In [30]:
users_lvl_2_pred_als.head()

,user_id,predictions,prec,rec,predictions_tfidf,prec_tfidf,rec_tfidf,predictions_bm25,prec_bm25,rec_bm25
0,338,"[1037863, 981760, 845208, 904360, 1082185, 113...",0.4,0.027778,"[845208, 923746, 896613, 904360, 1068292, 1037...",0.0,0.000000,"[871611, 819840, 1084036, 1026118, 1025650, 84...",0.2,0.013889
1,2120,"[1082185, 1106523, 981760, 995242, 1029743, 10...",0.0,0.000000,"[1082185, 1029743, 995242, 981760, 1127831, 11...",0.0,0.000000,"[1082185, 981760, 849843, 1041476, 995055, 113...",0.0,0.000000
2,2324,"[995242, 1029743, 1082185, 5978656, 981760, 91...",0.6,0.075000,"[1082185, 1136257, 1039156, 1027569, 5978656, ...",0.6,0.075000,"[10356272, 878611, 967760, 1108894, 942045, 96...",0.2,0.025000
3,514,"[866211, 878996, 1127831, 1082185, 1005186, 10...",0.6,0.093750,"[866211, 1105488, 1005186, 965267, 1024306, 87...",0.4,0.062500,"[1126786, 1082185, 878996, 923169, 835595, 639...",0.8,0.125000
4,1762,"[1098066, 1136257, 826249, 1068719, 1084331, 1...",0.0,0.000000,"[1096036, 1084331, 1110572, 1003188, 1044078, ...",0.2,0.012048,"[5569471, 1029743, 1048200, 868548, 1096036, 8...",0.2,0.012048


### Строим классификатор

In [120]:
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']

users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: get_pred_als(recommender, user_train_list, item_list, x, num=num))

In [121]:
users_lvl_2.head()

,user_id,candidates
0,2070,"[1029743, 883404, 1082185, 908531, 1127831, 10..."
1,2021,"[951590, 1082185, 981760, 1044078, 826249, 844..."
2,1753,"[1005186, 879755, 1106523, 1085604, 1133018, 9..."
3,2120,"[1082185, 1106523, 981760, 995242, 1029743, 10..."
4,1346,"[1082185, 1127831, 1058997, 1126899, 866211, 8..."


In [122]:
users_lvl_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2154 entries, 0 to 2153
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     2154 non-null   int64 
 1   candidates  2154 non-null   object
dtypes: int64(1), object(1)
memory usage: 33.8+ KB


In [123]:
df=pd.DataFrame({'user_id':users_lvl_2.user_id.values.repeat(len(users_lvl_2.candidates[0])),
                 'item_id':np.concatenate(users_lvl_2.candidates.values)})

In [124]:
targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
targets_lvl_2['target'] = 1  # тут только покупки 

targets_lvl_2 = df.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

targets_lvl_2['target'].fillna(0, inplace= True)

In [125]:
targets_lvl_2.shape

(94223, 3)

In [126]:
targets_lvl_2 = targets_lvl_2.merge(item_features, on='item_id', how='left') #
targets_lvl_2 = targets_lvl_2.merge(user_features, on='user_id', how='left') #

targets_lvl_2.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,bill_avg,week_last_bill,items_per_week
0,2070,1029743,0.0,69,GROCERY,Private,FLUID MILK PRODUCTS,FLUID MILK WHITE ONLY,1 GA,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,2.891039,95.0,29.814815
1,2070,883404,1.0,69,GROCERY,Private,BAKED BREAD/BUNS/ROLLS,MAINSTREAM WHITE BREAD,20 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,2.891039,95.0,29.814815


In [127]:
targets_lvl_2.shape

(94223, 19)

In [128]:
targets_lvl_2.fillna(0, inplace=True)

In [129]:
X_train = targets_lvl_2.drop('target', axis=1)
y_train = targets_lvl_2[['target']]

In [130]:
cat_feats = X_train.columns[2:].tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')

cat_feats

['manufacturer',
 'department',
 'brand',
 'commodity_desc',
 'sub_commodity_desc',
 'curr_size_of_product',
 'age_desc',
 'marital_status_code',
 'income_desc',
 'homeowner_desc',
 'hh_comp_desc',
 'household_size_desc',
 'kid_category_desc',
 'bill_avg',
 'week_last_bill',
 'items_per_week']

In [131]:
lgb = LGBMClassifier(objective='binary', 
                     n_estimators=1500,
                     learning_rate=0.008,
                     max_depth=7, 
                     verbose=1,
                     categorical_column=cat_feats)

hist = lgb.fit(X_train, y_train, 
               eval_set=[(X_train, y_train)],
               eval_metric=['logloss'],
               early_stopping_rounds=50,
               verbose=0)

train_preds = lgb.predict_proba(X_train)

In [132]:
hist.evals_result_['training']['binary_logloss'][-10:]

[0.3797458828180062,
 0.3797249579954718,
 0.3796862092747186,
 0.3796616880164342,
 0.37964265391768953,
 0.37962456171089826,
 0.3796061531401314,
 0.379572166790896,
 0.3795370841628755,
 0.37948243502924556]

In [133]:
classifer_prediction = X_train[['user_id', 'item_id']]
classifer_prediction["pred"] = train_preds[:, 1]
classifer_prediction.tail()

,user_id,item_id,pred
94218,1745,1038217,0.019470
94219,1745,985119,0.018644
94220,1745,1043590,0.016858
94221,1745,854852,0.016256
94222,1745,9526410,0.003377


In [134]:
res_val = classifer_prediction.loc[classifer_prediction["pred"]>0.5, ["user_id", "item_id"]]
res_val.shape

(9708, 2)

In [135]:
pred_lvl_2 = res_val.groupby('user_id')['item_id'].unique().reset_index()
pred_lvl_2.columns=['user_id', 'pred']
pred_lvl_2.head()

,user_id,pred
0,1,"[995242, 1082185, 1029743, 940947]"
1,7,"[1082185, 1106523, 1126899]"
2,8,[1082185]
3,13,"[840361, 5978648, 1126899, 1029743, 5978656, 9..."
4,17,"[1029743, 995242, 1082185, 862349]"


In [136]:
result_lvl_2 = result_lvl_2.merge(pred_lvl_2, on='user_id', how='left')
result_lvl_2.head()

,user_id,actual,pred
0,1,"[999999, 856942, 865456, 889248, 907957, 91419...","[995242, 1082185, 1029743, 940947]"
1,3,"[835476, 999999, 872021, 909638, 920626, 95815...",NaN
2,6,"[999999, 946489, 1006718, 1104227, 1108624, 82...",NaN
3,7,"[840386, 999999, 898068, 909714, 993838, 10031...","[1082185, 1106523, 1126899]"
4,8,"[835098, 872137, 910439, 924610, 992977, 99999...",[1082185]


In [137]:
user_pred_nan = result_lvl_2.loc[result_lvl_2["pred"] != result_lvl_2["pred"], 'user_id'].values
len(user_pred_nan)

1329

In [138]:
set_tr_1 = set(data_train_lvl_1.user_id.unique())
len(set_tr_1)

2498

In [139]:
user_list = set(user_pred_nan).intersection(set_tr_1)
len(user_list)

1328

In [140]:
result_lvl_2.loc[result_lvl_2["user_id"].isin(user_list), "pred"] =\
result_lvl_2.loc[result_lvl_2["user_id"].isin(user_list), "user_id"].apply(lambda x: recommender.get_als_recommendations(x, rec_num=k_n))
result_lvl_2.head()

,user_id,actual,pred
0,1,"[999999, 856942, 865456, 889248, 907957, 91419...","[995242, 1082185, 1029743, 940947]"
1,3,"[835476, 999999, 872021, 909638, 920626, 95815...","[1092026, 1106523, 1053690, 844165, 5568378]"
2,6,"[999999, 946489, 1006718, 1104227, 1108624, 82...","[866211, 1023720, 1024306, 878996, 1127831]"
3,7,"[840386, 999999, 898068, 909714, 993838, 10031...","[1082185, 1106523, 1126899]"
4,8,"[835098, 872137, 910439, 924610, 992977, 99999...",[1082185]


In [141]:
result_lvl_2.loc[result_lvl_2["user_id"] == 1984]

,user_id,actual,pred
1622,1984,"[999999, 847344, 859237, 1064380, 1074516, 122...",NaN


In [142]:
popularity = recommender.prep_data.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
top_5 = popularity.sort_values('n_sold', ascending=False).head(6).item_id.tolist()[1:6]
top_5

[1082185, 995242, 1029743, 1133018, 981760]

In [143]:
result_lvl_2.loc[result_lvl_2["user_id"] == 1984, "pred"] =\
        result_lvl_2.loc[result_lvl_2["user_id"] == 1984, "user_id"].apply(lambda x: top_5)

In [144]:
result_lvl_2.loc[result_lvl_2["user_id"] == 1984]

,user_id,actual,pred
1622,1984,"[999999, 847344, 859237, 1064380, 1074516, 122...","[1082185, 995242, 1029743, 1133018, 981760]"


In [145]:
len(result_lvl_2.loc[result_lvl_2["user_id"] == 1984, "pred"].values[0])

5

In [146]:
res = []
res.extend(result_lvl_2.loc[result_lvl_2["user_id"] == 1984, "pred"].values[0])
res

[1082185, 995242, 1029743, 1133018, 981760]

In [147]:
def fill_if_not_5(recommender, x):
    res = []
    res.extend(result_lvl_2.loc[result_lvl_2["user_id"] == x, "pred"].values[0])
    pred_num = len(result_lvl_2.loc[result_lvl_2["user_id"] == x, "pred"].values[0])
    if pred_num < 5:
        res.extend(recommender.get_als_recommendations(x, rec_num=5-pred_num))
    return res

In [148]:
result_lvl_2["pred"] = result_lvl_2["user_id"].apply(lambda x: fill_if_not_5(recommender, x))

In [149]:
result_lvl_2.head()

,user_id,actual,pred
0,1,"[999999, 856942, 865456, 889248, 907957, 91419...","[995242, 1082185, 1029743, 940947, 965766]"
1,3,"[835476, 999999, 872021, 909638, 920626, 95815...","[1092026, 1106523, 1053690, 844165, 5568378]"
2,6,"[999999, 946489, 1006718, 1104227, 1108624, 82...","[866211, 1023720, 1024306, 878996, 1127831]"
3,7,"[840386, 999999, 898068, 909714, 993838, 10031...","[1082185, 1106523, 1126899, 1082185, 1127831]"
4,8,"[835098, 872137, 910439, 924610, 992977, 99999...","[1082185, 840361, 5569230, 1005186, 883404]"


In [150]:
result_lvl_2["rec"] = result_lvl_2['user_id'].apply(\
        lambda x: precision_at_k(result_lvl_2.loc[result_lvl_2["user_id"] == x, "pred"].values[0], 
                                 list(result_lvl_2.loc[result_lvl_2["user_id"] == x, "actual"].values[0]), 
                                 k=5))

result_lvl_2['rec'].mean()

0.2531831537708103

### LightGBM Ranker

In [62]:
X_train.shape

(48545, 18)

In [63]:
X_train['num'] = np.arange(len(X_train))

In [64]:
gr = X_train[['user_id', 'num']].groupby('user_id', sort=False).max()['num'].values

In [65]:
grs = np.concatenate((np.array([gr[0]+1]), gr[1:] - gr[:-1]))

In [66]:
X_train.shape, y_train.shape, np.sum(grs)

((48545, 19), (48545, 1), 48545)

In [67]:
from lightgbm import LGBMRanker

lgb_params = { 
    'objective':'lambdarank',
    'boosting_type': 'gbdt',
    'n_estimators': 1000,
    'learning_rate': 0.08,
    'max_depth': 5,
    'categorical_column': cat_feats,
    'random_state': 27,
    'verbose': 0,
    'is_unbalance': True
}

In [68]:
lgb_rn = LGBMRanker(**lgb_params, silent=False)

eval_h = lgb_rn.fit(X_train, y_train, group=grs,
                    eval_set=[(X_train, y_train)], eval_group=[grs], 
                    eval_metric=['ndcg'],
                    eval_at=[5, 10 ], 
                    early_stopping_rounds=50, 
                    verbose=0)

In [69]:
eval_h.evals_result_['training']['ndcg@5'][:3], eval_h.evals_result_['training']['ndcg@5'][-3:]

([0.6276553608168751, 0.6276553608168751, 0.6352907909088611],
 [0.8364698088054933, 0.8364261946941001, 0.8364261946941001])

In [70]:
rank_preds = lgb_rn.predict(X_train)

In [71]:
ranker_prediction = X_train[['user_id', 'item_id']]
ranker_prediction["pred"] = rank_preds
ranker_prediction.head()

,user_id,item_id,pred
0,2070,1029743,-0.917832
1,2070,883404,-0.274296
2,2070,1082185,-0.462071
3,2070,908531,-0.647724
4,2070,1127831,-1.661238


In [72]:
ranker_prediction = ranker_prediction.drop_duplicates()

In [73]:
ranker_prediction.sort_values(by="pred", inplace=True, ascending=False)
ranker_prediction.head()

,user_id,item_id,pred
5574,1383,1077975,2.525370
37522,142,1093966,1.967510
27013,1914,894439,1.772266
38722,183,8090440,1.756711
6601,27,7409957,1.655706


In [74]:
ranked_res = ranker_prediction.groupby('user_id')['item_id'].unique().reset_index()   
ranked_res

,user_id,item_id
0,1,"[940947, 1082185, 995242, 1029743, 979707, 856..."
1,2,"[1106523, 1082185, 1053690, 1092026, 1037840, ..."
2,4,"[883932, 1029743, 1082185, 962229, 1106523, 99..."
3,6,"[962568, 1082185, 995242, 845208, 849843, 9817..."
4,7,"[1082185, 1106523, 1126899, 1122358, 1058997, ..."
...,...,...
2149,2496,"[1106523, 979707, 1056509, 981760, 916122, 826..."
2150,2497,"[1029743, 900802, 845208, 1082185, 1081177, 89..."
2151,2498,"[1070820, 1053690, 1126899, 1092026, 1082185, ..."
2152,2499,"[5568378, 5569327, 883404, 1070820, 1106523, 1..."


In [75]:
result_lvl_2 = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_2.columns=['user_id', 'actual']

result_lvl_2 = result_lvl_2.merge(ranked_res, on='user_id', how='left')
result_lvl_2

,user_id,actual,item_id
0,1,"[999999, 856942, 865456, 889248, 907957, 91419...","[940947, 1082185, 995242, 1029743, 979707, 856..."
1,3,"[835476, 999999, 872021, 909638, 920626, 95815...",NaN
2,6,"[999999, 946489, 1006718, 1104227, 1108624, 82...","[962568, 1082185, 995242, 845208, 849843, 9817..."
3,7,"[840386, 999999, 898068, 909714, 993838, 10031...","[1082185, 1106523, 1126899, 1122358, 1058997, ..."
4,8,"[835098, 872137, 910439, 924610, 992977, 99999...","[1082185, 1005186, 1029743, 923746, 1070820, 1..."
...,...,...,...
2037,2496,[999999],"[1106523, 979707, 1056509, 981760, 916122, 826..."
2038,2497,"[1016709, 9835695, 999999, 845294, 871756, 873...","[1029743, 900802, 845208, 1082185, 1081177, 89..."
2039,2498,"[999999, 901776, 914190, 958382, 972437, 10398...","[1070820, 1053690, 1126899, 1092026, 1082185, ..."
2040,2499,"[867188, 999999, 902396, 914190, 951590, 95813...","[5568378, 5569327, 883404, 1070820, 1106523, 1..."


In [76]:
user_pred_nan = result_lvl_2.loc[result_lvl_2["item_id"] != result_lvl_2["item_id"], 'user_id'].values
len(user_pred_nan)

126

In [77]:
set_tr_1 = set(data_train_lvl_1.user_id.unique())
len(set_tr_1)

2498

In [78]:
user_list = set(user_pred_nan).intersection(set_tr_1)
len(user_list)

126

In [79]:
result_lvl_2.loc[result_lvl_2["user_id"].isin(user_list), "item_id"] =\
result_lvl_2.loc[result_lvl_2["user_id"].isin(user_list), "user_id"].apply(lambda x: recommender.get_als_recommendations(x, rec_num=k_n))
result_lvl_2.head()

,user_id,actual,item_id
0,1,"[999999, 856942, 865456, 889248, 907957, 91419...","[940947, 1082185, 995242, 1029743, 979707, 856..."
1,3,"[835476, 999999, 872021, 909638, 920626, 95815...","[1092026, 1106523, 1053690, 844165, 5568378]"
2,6,"[999999, 946489, 1006718, 1104227, 1108624, 82...","[962568, 1082185, 995242, 845208, 849843, 9817..."
3,7,"[840386, 999999, 898068, 909714, 993838, 10031...","[1082185, 1106523, 1126899, 1122358, 1058997, ..."
4,8,"[835098, 872137, 910439, 924610, 992977, 99999...","[1082185, 1005186, 1029743, 923746, 1070820, 1..."


In [80]:
result_lvl_2.loc[result_lvl_2["user_id"] == 1984]

,user_id,actual,item_id
1622,1984,"[999999, 847344, 859237, 1064380, 1074516, 122...","[1126899, 1106523, 1029743, 1082185, 995242, 1..."


In [81]:
result_lvl_2["rec"] = result_lvl_2['user_id'].apply(\
        lambda x: precision_at_k(result_lvl_2.loc[result_lvl_2["user_id"] == x, "item_id"].values[0], 
                                 list(result_lvl_2.loc[result_lvl_2["user_id"] == x, "actual"].values[0]), 
                                 k=5))

result_lvl_2['rec'].mean()

0.28961802154750177